In [11]:
# loading libraries for data manipulation
import numpy as np
import pandas as pd

# loading libraries for data visualization
import matplotlib.pyplot as plt
from plotnine import *

# other helpers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import tensorflow and keras packages
import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

Here's a single neuron with some input, weights, and a bias value. We can also calculate the weighted sum before the value is passed to an activation function. 

In [ ]:
x = np.array([0.8, 1.2])   # input features
w = np.array([0.6, -0.1])  # weights
b = 0.1                    # bias
z = np.dot(w, x) + b
print(z)

Activation functions create a non-linear output (excluding step and linear activation) for which a gradient can be calculated. 
Here, we will plot sigmoid, tanh, and ReLU activation function output. 

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

x_values = ...

df_activations = pd.DataFrame({
    'x':x_values,
    'sigmoid' : sigmoid(x_values),
    'tanh' : tanh(x_values),
    'relu' : relu(x_values) 
})

(
    ggplot(df_activations)+
    geom_line(aes(x='x', y='sigmoid'), color='blue')+
    geom_line(aes(x='x', y='tanh'), color='orange')+
    geom_line(aes(x='x', y='relu'), color='green')+
    labs(title='Activation Functions', x='Input', y='Activation')+
    theme_minimal()
)

Let's train a linear regression model using the building blocks of a neural network implementation. 

When building a neural network, think of the following key items:
- The structure of your model (how many nodes/layers? which activation function to use?)
- The loss function of your model
- The optimizer you will use to train the model (more to come on this)

The input and output layers are determined by your feature space (size and type of input) and the target (what form of output you expect). For a regression problem, your output layer will have a single node. For a multi-class classification, it will be n nodes representing n classes. 

In [ ]:
# first we load our data
df = pd.read_csv("https://raw.githubusercontent.com/cristobalvch/Spotify-Machine-Learning/refs/heads/master/data/data_moods.csv")
df.head()

In [13]:
# let's define our feature space and target
# we will predict 'energy' using the other features

features = ['danceability','popularity','length','acousticness','energy','instrumentalness','liveness','valence','loudness','speechiness','tempo']

X = df[features]
y = df['energy']

In [ ]:
# let's also standardize our feature space

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Building blocks of a neural network in Tensorflow/Keras


#### Model Structure

| **Component**   | **Function / Parameter**                                | **Description**                                              |
| --------------- | ------------------------------------------------------- | ------------------------------------------------------------ |
| **Model**       | `tf.keras.Sequential()` or subclass of `tf.keras.Model` | Container that defines the model architecture.               |
| **Layers**      | `tf.keras.layers.Dense(units, activation)`              | Fully connected layers used to build the model.              |
| **Units**       | `units=...`                                             | Number of neurons in a layer.                                |
| **Activation**  | `activation='relu'`, `'sigmoid'`, `'softmax'`           | Function applied to layer output to introduce non-linearity. |
| **Input Shape** | `input_shape=(...)`                                     | Shape of input data (required in the first layer).           |

#### Compiling

| **Component**     | **Function / Parameter**                    | **Description**                                                          |
| ----------------- | ------------------------------------------- | ------------------------------------------------------------------------ |
| **Compile**       | `model.compile()`                           | Prepares the model for training by setting loss, optimizer, and metrics. |
| **Loss Function** | `loss='mse'`, `'binary_crossentropy'`, etc. | Measures the difference between predictions and actual values.           |
| **Optimizer**     | `optimizer='sgd'`, `'adam'`, etc.           | Algorithm to update weights based on gradients.                          |
| **Metrics**       | `metrics=['accuracy']`, etc.                | List of metrics to evaluate during training/testing.                     |

#### Training and Evaluation

| **Component**  | **Function / Parameter**         | **Description**                                         |
| -------------- | -------------------------------- | ------------------------------------------------------- |
| **Fit**        | `model.fit(X, y, epochs=...)`    | Trains the model using input data and labels.           |
| **Epochs**     | `epochs=...`                     | Number of complete passes through the training data.    |
| **Batch Size** | `batch_size=...`                 | Number of samples processed before model weight update. |
| **Evaluate**   | `model.evaluate(X_test, y_test)` | Tests model performance on new data.                    |
| **Predict**    | `model.predict(X_new)`           | Uses the model to make predictions on new data.         |

#### Other Blocks

| **Component**     | **Function / Parameter**                     | **Description**                                                         |
| ----------------- | -------------------------------------------- | ----------------------------------------------------------------------- |
| **Model Weights** | `model.get_weights()`, `model.set_weights()` | Access or modify model parameters directly.                             |
| **Callbacks**     | `callbacks=[...]`                            | Tools like early stopping, learning rate scheduling, and checkpointing. |


In [ ]:
# lets build a simple neural network: one neural, no activation function
model = tf.keras.Sequential([...])

# compile the model

# train the model

During each epoch:  (actually done in batches but we will discuss this later)
- 1. Forward Pass: model makes prediction on X_train observations
- 2. Loss Computation: MSE is calculated between predicted and true values
- 3. Backpropagation: gradients are computed with respect to weights
- 4. SDG Optimizer: updates weights
- 5. Repeat

In [ ]:
# print learned weights and bias
weights, bias = ...
print("Weights:", weights.flatten())
print("Bias:", bias)

In [ ]:
# evaluate model
mse = model.evaluate(X_train, y_train, verbose=0)
print("Mean Squared Error on training set:", mse)

mse = model.evaluate(X_test, y_test, verbose=0)
print("Mean Squared Error on test set:", mse)

Let's build a deep neural network - that means adding some hidden layers! Recall that if no activation function is used then all layers combine into a single linear combination. 

In [ ]:
# build a deep neural network with multiple layers and activation functions

deep_model = tf.keras.Sequential([...])

# compile the model

# train the model


In [ ]:

# evaluate deep_model
mse = deep_model.evaluate(X_train, y_train, verbose=0)
print("Mean Squared Error on training set:", mse)

mse = deep_model.evaluate(X_test, y_test, verbose=0)
print("Mean Squared Error on test set:", mse)

In [51]:
# display the model summary